In [2]:
import numpy as np
import pandas as pd
from foundation import *

In [3]:
def obj_felipe_lucas(x, args):

    # Argumentos
    df = args[0].copy()
    n_comb = args[1]
    f_ck = args[2]
    h_z = args[3]
    n_fun = df.shape[0]

    # Variáveis de projeto
    x = np.asarray(x).reshape(n_fun, 2)   
    df['h_x (m)'] = x[:, 0]
    df['h_y (m)'] = x[:, 1]
    df['h_z (m)'] = h_z

    # Volume
    df['volume (m3)'] = df['h_x (m)'] * df['h_y (m)'] * df['h_z (m)']

    # Rótulo das combinações
    labels_comb = [f'c{i}' for i in range(1, n_comb + 1)]

    # Checagem punção
    for i in labels_comb:
        aux = f'{i}'
        df[[f'tau_sd2 - {aux}', f'tau_rd2 - {aux}', f'u_rd2 - {aux}', f'g_rd2 - {aux}', f'k_e - {aux}', f'g_ed - {aux}', f'tau_rd1 - {aux}', f'u_rd1 - {aux}', f'kx - {aux}', f'ky - {aux}', f'w_px - {aux}', f'w_py - {aux}', f'tau_sd1 - {aux}', f'g_rd1 - {aux}']] = df.apply(lambda row: verificacao_puncao_sapata(row['h_z (m)'], f_ck, row['ap (m)'], row['bp (m)'], row[f'Fz-{aux}'], row[f'Mx-{aux}'], row[f'My-{aux}']), axis=1, result_type='expand')
    df['g punção secao C'] = df[[f'g_rd2 - {i}' for i in labels_comb]].max(axis=1)
    df['g escala punção'] = df[[f'g_ed - {i}' for i in labels_comb]].max(axis=1)
    df['g punção secao Clinha'] = df[[f'g_rd1 - {i}' for i in labels_comb]].max(axis=1)

    
    # Volume final com penalizações
    df['volume final (m3)'] = df['volume (m3)'] +  df['g punção secao C'].clip(lower=0) * 1E6 + df['g escala punção'].clip(lower=0) * 1E6 + df['g punção secao Clinha'].clip(lower=0) * 1E6
    of = df['volume final (m3)'].sum()

    return of

In [4]:
df = pd.read_excel('DADOS_TESTE_PUNCAO.xlsx')
n_comb = 3
f_ck = 30000
h_z = 0.2
args = args=(df, n_comb, f_ck, h_z)
x = [0.6, 0.6]

In [5]:
of=obj_felipe_lucas(x, args)
print(of)

520501.50037568546


In [6]:
i=1 #para rodar na combinação 1

row = df.loc[0]

tau_sd2, tau_rd2, u_rd2, g_rd2, k_e, g_ed, tau_rd1, u_rd1, kx, ky, w_px, w_py, tau_sd1, g_rd1 = verificacao_puncao_sapata(
    h_z,
    f_ck=f_ck,
    a_p=row['ap (m)'],
    b_p=row['bp (m)'],
    f_zk=row[f'Fz-c{i}'],
    m_xk=row[f'Mx-c{i}'],
    m_yk=row[f'My-c{i}'],
    sigma_cp=0.0,
    cob=0.04
)

dados = {
    "Parâmetro": [
        "tau_sd2 (kPa)", "tau_rd2 (kPa)", "u_rd2 (m)", "g_rd2",
        "k_e", "g_ed",
        "tau_rd1 (kPa)", "u_rd1 (m)",
        "kx", "ky",
        "w_px (m³)", "w_py (m³)",
        "tau_sd1 (kPa)", "g_rd1"
    ],
    "Valor": [
        tau_sd2, tau_rd2, u_rd2, g_rd2,
        k_e, g_ed,
        tau_rd1, u_rd1,
        kx, ky,
        w_px, w_py,
        tau_sd1, g_rd1
    ]
}

df_saida = pd.DataFrame(dados)

print(f"\n=== VERIFICAÇÃO À PUNÇÃO – SAPATA 0 | COMBINAÇÃO c{i} ===\n")
print(df_saida.to_string(index=False, float_format="%.4f"))



=== VERIFICAÇÃO À PUNÇÃO – SAPATA 0 | COMBINAÇÃO c1 ===

    Parâmetro     Valor
tau_sd2 (kPa) 1715.0000
tau_rd2 (kPa) 5091.4286
    u_rd2 (m)    1.5000
        g_rd2   -0.6632
          k_e    2.0000
         g_ed    0.0000
tau_rd1 (kPa)  645.4617
    u_rd1 (m)    3.5106
           kx    0.7000
           ky    0.4500
    w_px (m³)    1.3223
    w_py (m³)    1.1372
tau_sd1 (kPa) 1106.9504
        g_rd1    0.7150


In [7]:
rho = rho_minimo_fck(f_ck)
print(f"\n=== CÁLCULO DA ARMADURA MÍNIMA PARA FCK = {f_ck} kPa ===")
print(f"Densidade mínima de armadura: {rho:.4f} %")


=== CÁLCULO DA ARMADURA MÍNIMA PARA FCK = 30000 kPa ===
Densidade mínima de armadura: 0.1500 %
